##GLCM

복수의 image로부터 GLCM를 이용하여 imaging feature들을 계산하고, 
결과를 엑셀파일로 저장하는 과정입니다.

필요한 외부 라이브러리는 
skiimage, numpy, pandas, xlsxwriter 
입니다.

의료영상연구에서는 대개 ROI에서 GLCM을 계산합니다. 
여기서는 crop된 영상들을 입력으로 받도록 하였습니다.

GLCM의 계산과 관련된 구체적인 내용은 아래를 참고해주세요.
http://scikit-image.org/docs/dev/auto_examples/features_detection/plot_glcm.html

In [1]:
from skimage.feature import greycomatrix, greycoprops
from skimage import data
from skimage.io import imread


In [2]:
import os
pathimg = "./cucumber"
imgfiles = []
for dirName, subdirList, fileList in os.walk(pathimg):
    for filename in fileList:
        if ".png" in filename.lower():
            imgfiles.append(os.path.join(dirName,filename))

이미지파일 리스트를 만드는 과정입니다. 
cucumber라는 폴더 안에 png 포맷으로 이미지가 저장되어 있는 경우에 대한 예시입니다.
필요에 따라서 본인의 폴더명과 이미지 확장자로 바꿔 주시면 됩니다.

In [3]:
imgfiles

['./cucumber\\c_0001.png',
 './cucumber\\c_0002.png',
 './cucumber\\c_0003.png',
 './cucumber\\c_0004.png',
 './cucumber\\c_0005.png',
 './cucumber\\c_0006.png',
 './cucumber\\c_0007.png',
 './cucumber\\c_0008.png']

In [4]:
import numpy as np
g=np.zeros((len(imgfiles),6),float)
for n in range(len(imgfiles)):
    img=imread(imgfiles[n])
    glcm = greycomatrix(img,[5], [0], 256, symmetric=True, normed=True)
    g[n,0]=greycoprops(glcm, 'contrast')
    g[n,1]=greycoprops(glcm, 'dissimilarity')
    g[n,2]=greycoprops(glcm, 'homogeneity')
    g[n,3]=greycoprops(glcm, 'energy')
    g[n,4]=greycoprops(glcm, 'correlation')
    g[n,5]=greycoprops(glcm, 'ASM')

여기서 주의할 부분은 greycomatrix 함수 안에 있는 값들입니다.
[5], [0], ... 들을 사용하였는데 본인이 원하는 파라미터로 바꾸어 줄 수 있습니다. 
여러 경우에 대해 한 번에 계산을 할 경우, greycoprops의 값들도 복수가 될 수 있으므로 수정이 필요합니다. 

자세한 옵션에 대한 설명은 아래를 참고해 주세요.
http://scikit-image.org/docs/dev/api/skimage.feature.html?highlight=greycomatrix#skimage.feature.greycomatrix
    

In [5]:
import pandas as pd
import xlsxwriter

df0 = pd.DataFrame({'filename': imgfiles})
df1 = pd.DataFrame({'contrast': g[:,0]})
df2 = pd.DataFrame({'dissimilarity': g[:,1]})
df3 = pd.DataFrame({'homoheneity': g[:,2]})
df4 = pd.DataFrame({'energy': g[:,3]})
df5 = pd.DataFrame({'correlation': g[:,4]})
df6 = pd.DataFrame({'ASM': g[:,5]})

writer = pd.ExcelWriter('cucumber_GLCM.xlsx', engine='xlsxwriter')

df0.to_excel(writer, sheet_name='Sheet1')
df1.to_excel(writer, sheet_name='Sheet1', startcol=2, index=False)
df2.to_excel(writer, sheet_name='Sheet1', startcol=3, index=False)
df3.to_excel(writer, sheet_name='Sheet1', startcol=4, index=False)
df4.to_excel(writer, sheet_name='Sheet1', startcol=5, index=False)
df5.to_excel(writer, sheet_name='Sheet1', startcol=6, index=False)
df6.to_excel(writer, sheet_name='Sheet1', startcol=7, index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

pandas와 xlsxwriter를 이용해서 결과를 excel파일로 저장하는 과정입니다.
만들어진 excel파일의 내용과 대조해서 코드를 보면 쉽게 이해할 수 있을 것입니다.